In [ ]:
from transformers import pipeline
model = "siebert/sentiment-roberta-large-english"

analysis = pipeline(task="sentiment-analysis",model=model) 
print(analysis("Something ain't right"))

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
#https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest?text=Covid+cases+are+increasing+fast%21
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

analysis = pipeline(task="sentiment-analysis",model=model,tokenizer=tokenizer) 
print(analysis("When twitter spaces ?"))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'neutral', 'score': 0.9254334568977356}]


In [6]:
import twint
import nest_asyncio
nest_asyncio.apply()

from transformers import pipeline
model = "siebert/sentiment-roberta-large-english"
analysis = pipeline(task="sentiment-analysis",model=model) 

# Configure
c = twint.Config()
c.Search = "TWitter"
c.Limit = 200
c.Lang = "en"
c.Pandas = True

# Run
twint.run.Search(c)
Tweets_df = twint.storage.panda.Tweets_df

#Sub selects english tweets
en_df = Tweets_df.query("language == 'en'")
print("Length of tweets in English: " + str(len(en_df)))

#Drop rows that are duplicated and from the same user
en_df.drop_duplicates(subset=['tweet','user_id'])

#Only select columns that are used
df = en_df[["tweet", "nlikes","nretweets","nreplies"]]

#Calculate traction of tweet
df["traction"]= df["nlikes"] * 2 + df["nreplies"] * 3 + df["nretweets"] * 4

#Sorts the dataframe according to the most number of likes
sorted = df.sort_values(by=['traction'], ascending=False)
sorted

1599040137349906432 2022-12-03 21:57:32 +0800 <Daniell76860413> I'm single, can I get a husband on Twitter?  #深圳 #深圳线下  https://t.co/STMb368Lzt
1599040135605387264 2022-12-03 21:57:31 +0800 <thechloeparish> @carissaxox_ @theGaruboBus @DaYambayg @Lillipie101 Girl. I’m going to say this one more time. Just stop. Didn’t Matt ask ppl in his community to not do EXACTLY what you’re doing? You’re doing more damage than good. Maybe log off Twitter for a few days a take a breather. Or don’t. Bc it’s pretty hilarious 😂
1599040135366316032 2022-12-03 21:57:31 +0800 <Jessica68044049> I'm single, can I get a husband on Twitter?  #南京 #线下  https://t.co/pq7184HmJe
1599040135118876673 2022-12-03 21:57:31 +0800 <nanohachan0524> 私とね一緒に居る子は不幸になる傾向があるんじゃないかってぐらい最近FFさんが傷ついててさ。 たまたまだとは私も思ってる…けど。私から離れたFFさんが凄く幸せそうでさ。これが何十人もだよ…？もう信じざるおえなくてって悩んでいるんだったらもういっそのことTwitter辞めようかなって( ˙꒳​˙  )
1599040133927698432 2022-12-03 21:57:31 +0800 <nensma_toni6> 3人の中での流行りがTwitterなのかわいい (●●)3人でやったよ  #ネクジェネ
1599040133852008449 2022-

<ipython-input-6-b93fa39f9367>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["traction"]= df["nlikes"] * 2 + df["nreplies"] * 3 + df["nretweets"] * 4


,tweet,nlikes,nretweets,nreplies,traction
628,Rollin' into Twitter every morning and blockin...,17,1,0,38
655,I don’t know what Trump is so mad about. I bet...,4,1,0,12
707,Should I make a twitter circle?,4,0,2,14
615,I joined Twitter in May 2020. Look at my profi...,3,0,0,6
1138,When twitter spaces ?,3,0,0,6
...,...,...,...,...,...
415,"I'm single, can I get a husband on Twitter? #...",0,0,0,0
416,@Ron06212876 @TomFitton @keepitwilder @elonmus...,0,0,0,0
417,@JennaEllisEsq @elonmusk @hodgetwins @micsolan...,0,0,0,0
418,Feel free to identify what law was passed by C...,0,0,0,0


In [8]:
sorted["sentiment"] = sorted["tweet"].apply(analysis)

In [9]:
sorted

,tweet,nlikes,nretweets,nreplies,traction,sentiment
628,Rollin' into Twitter every morning and blockin...,17,1,0,38,"[{'label': 'NEGATIVE', 'score': 0.999027729034..."
655,I don’t know what Trump is so mad about. I bet...,4,1,0,12,"[{'label': 'NEGATIVE', 'score': 0.998800992965..."
707,Should I make a twitter circle?,4,0,2,14,"[{'label': 'NEGATIVE', 'score': 0.944785714149..."
615,I joined Twitter in May 2020. Look at my profi...,3,0,0,6,"[{'label': 'NEGATIVE', 'score': 0.998424410820..."
1138,When twitter spaces ?,3,0,0,6,"[{'label': 'NEGATIVE', 'score': 0.989874362945..."
...,...,...,...,...,...,...
415,"I'm single, can I get a husband on Twitter? #...",0,0,0,0,"[{'label': 'NEGATIVE', 'score': 0.975065588951..."
416,@Ron06212876 @TomFitton @keepitwilder @elonmus...,0,0,0,0,"[{'label': 'POSITIVE', 'score': 0.998225748538..."
417,@JennaEllisEsq @elonmusk @hodgetwins @micsolan...,0,0,0,0,"[{'label': 'NEGATIVE', 'score': 0.998326241970..."
418,Feel free to identify what law was passed by C...,0,0,0,0,"[{'label': 'NEGATIVE', 'score': 0.998207688331..."
